# Project 20 - Classification with autoencoder

In [1]:
from torchvision import datasets, transforms
import torch
from torch import nn, optim
from sklearn.metrics import accuracy_score

In [2]:
dataset_train = datasets.MNIST('Datasets/', train=True, download=True, transform=transforms.ToTensor())
loader_train = torch.utils.data.DataLoader(dataset_train, batch_size=256, shuffle=True)

In [3]:
dataset_val = datasets.MNIST('Datasets/', train=False, download=True, transform=transforms.ToTensor())
loader_val = torch.utils.data.DataLoader(dataset_val, batch_size=256, shuffle=True)

In [4]:
class autoencoder(nn.Module):
    def __init__(self):
        super().__init__()

        self.dense0 = nn.Linear(in_features=28*28, out_features=32)
        self.activation0 = nn.ReLU()

        self.dense1 = nn.Linear(32, 784)
        self.activation0 = nn.Sigmoid()

    def forward(self, X):
        X = self.dense0(X)
        X = self.activation0(X)

        X = self.dense1(X)
        X = self.activation0(X)

        return X
    

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [6]:
model = autoencoder().to(device)

In [7]:
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters())

In [11]:
for epoch in range(20):
    running_loss_train = 0

    for data in loader_train:
        model.train()
        
        inputs, _ = data
        inputs = inputs.to(device)

        inputs = inputs.view(-1, 28*28)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, inputs.view(*outputs.shape))
        loss.backward()

        optimizer.step()

        running_loss_train += loss.item()

        running_loss_val = 0

    for data in loader_val:
        model.eval()

        labels, _ = data
        labels = labels.to(device)

        labels = labels.view(-1, 28*28)

        outputs = model(labels)
        loss = criterion(outputs, labels.view(*outputs.shape))

        running_loss_val += loss.item()


    print(f'Epoch {epoch+1}/{10} - Loss Train: {running_loss_train/len(loader_train)} - Loss Val: {running_loss_val/len(loader_val)}')


Epoch 1/10 - Loss Train: 0.21899816856739368 - Loss Val: 0.21482280269265175
Epoch 2/10 - Loss Train: 0.2130847028595336 - Loss Val: 0.2101047184318304
Epoch 3/10 - Loss Train: 0.2081145866754207 - Loss Val: 0.20511040464043617
Epoch 4/10 - Loss Train: 0.20343511060197303 - Loss Val: 0.19990578405559062
Epoch 5/10 - Loss Train: 0.19792923248828725 - Loss Val: 0.19456506632268428
Epoch 6/10 - Loss Train: 0.19293773408899917 - Loss Val: 0.1897741425782442
Epoch 7/10 - Loss Train: 0.18894474645878406 - Loss Val: 0.1861878987401724
Epoch 8/10 - Loss Train: 0.1857147162264966 - Loss Val: 0.18345427624881266
Epoch 9/10 - Loss Train: 0.1831199366361537 - Loss Val: 0.18125321492552757
Epoch 10/10 - Loss Train: 0.18095261847719232 - Loss Val: 0.17903069853782655
Epoch 11/10 - Loss Train: 0.17907445823892634 - Loss Val: 0.17761519476771354
Epoch 12/10 - Loss Train: 0.17745132433607222 - Loss Val: 0.17576022669672967
Epoch 13/10 - Loss Train: 0.1760277803273911 - Loss Val: 0.17423576377332212
Epo

In [12]:
list(model.children())

[Linear(in_features=784, out_features=32, bias=True),
 Sigmoid(),
 Linear(in_features=32, out_features=784, bias=True)]

In [13]:
class encoder(nn.Module):
    def __init__(self):
        super().__init__()

        self.dense0 = list(model.children())[0]
        self.activation0 = list(model.children())[1]

    def forward(self, X):
        X = self.dense0(X)
        X = self.activation0(X)

        return X

In [14]:
model1 = encoder().to(device)

In [15]:
forecast_train = dataset_train.data.float() / 255
forecast_train = forecast_train.view(-1, 784)
forecast_train = forecast_train.to(device)

In [16]:
forecast_train.shape

torch.Size([60000, 784])

In [17]:
with torch.no_grad():
    forecast_train_codify = model1(forecast_train)

train_class = dataset_train.targets

In [18]:
forecast_train_codify.shape

torch.Size([60000, 32])

In [19]:
train_class

tensor([5, 0, 4,  ..., 5, 6, 8])

In [20]:
dataset_train_codify = torch.utils.data.TensorDataset(forecast_train_codify, train_class)
loader_train_codify = torch.utils.data.DataLoader(dataset_train_codify, batch_size=256, shuffle=True)

In [21]:
forecast_val = dataset_val.data.float() / 255
forecast_val = forecast_val.view(-1, 784)
forecast_val = forecast_val.to(device)

In [22]:
with torch.no_grad():
    forecast_val_codify = model1(forecast_val)

val_class = dataset_val.targets

In [23]:
forecast_val_codify.shape

torch.Size([10000, 32])

In [24]:
dataset_test_codify = torch.utils.data.TensorDataset(forecast_val_codify, val_class)
loader_test_codify = torch.utils.data.DataLoader(dataset_test_codify, batch_size=256, shuffle=True)

In [25]:
classify1 = nn.Sequential(
    nn.Linear(784, 397),
    nn.ReLU(),
    nn.Linear(397, 10),
    nn.LogSoftmax()
)

In [26]:
classify1 = classify1.to(device)

In [27]:
criterion = nn.NLLLoss()
optimizer = optim.Adam(classify1.parameters())

In [28]:
for epoch in range(20):
    running_loss_train = 0
    running_accuracy_train = 0

    for data in loader_train:
        classify1.train()

        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        
        inputs = inputs.view(-1, 784)
        optimizer.zero_grad()

        outputs = classify1(inputs)

        ps = torch.exp(outputs)
        _, top_class = ps.topk(1, dim=1)

        loss = criterion(outputs, labels)
        loss.backward()

        optimizer.step()

        running_loss_train += loss.item()
        running_accuracy_train += accuracy_score(labels.detach().cpu().numpy(), top_class.detach().cpu().numpy())

    running_loss_val = 0
    running_accuracy_val = 0

    for data in loader_val:
        classify1.eval()

        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        inputs = inputs.view(-1, 784)

        outputs = classify1(inputs)

        ps = torch.exp(outputs)
        _, top_class = ps.topk(1, dim=1)

        loss = criterion(outputs, labels)

        running_loss_val += loss.item()
        running_accuracy_val += accuracy_score(labels.detach().cpu().numpy(), top_class.detach().cpu().numpy())

    print(f'Epoch {epoch+1}/{20} - Loss Train: {running_loss_train/len(loader_train)} - Accuracy Train: {running_accuracy_train/len(loader_train)} - Loss Val: {running_loss_val/len(loader_val)} - Accuracy Val: {running_accuracy_val/len(loader_val)}')

/home/pcmoraes/Desktop/codigos/Neural-Networks/.venv/lib/python3.10/site-packages/torch/nn/modules/module.py:1532: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
/home/pcmoraes/Desktop/codigos/Neural-Networks/.venv/lib/python3.10/site-packages/torch/nn/modules/module.py:1532: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
/home/pcmoraes/Desktop/codigos/Neural-Networks/.venv/lib/python3.10/site-packages/torch/nn/modules/module.py:1532: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
/home/pcmoraes/Desktop/codigos/Neural-Networks/.venv/lib/python3.10/site-packages/torch/nn/modules/module.py:1532: UserWarning: Implicit dimension choice 

Epoch 1/20 - Loss Train: 0.41370934197243225 - Accuracy Train: 0.8911735372340426 - Loss Val: 0.2199347548186779 - Accuracy Val: 0.9369140625


/home/pcmoraes/Desktop/codigos/Neural-Networks/.venv/lib/python3.10/site-packages/torch/nn/modules/module.py:1532: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
/home/pcmoraes/Desktop/codigos/Neural-Networks/.venv/lib/python3.10/site-packages/torch/nn/modules/module.py:1532: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
/home/pcmoraes/Desktop/codigos/Neural-Networks/.venv/lib/python3.10/site-packages/torch/nn/modules/module.py:1532: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
/home/pcmoraes/Desktop/codigos/Neural-Networks/.venv/lib/python3.10/site-packages/torch/nn/modules/module.py:1532: UserWarning: Implicit dimension choice 

Epoch 2/20 - Loss Train: 0.17900648329486238 - Accuracy Train: 0.9482435726950355 - Loss Val: 0.14885277720168233 - Accuracy Val: 0.95556640625


/home/pcmoraes/Desktop/codigos/Neural-Networks/.venv/lib/python3.10/site-packages/torch/nn/modules/module.py:1532: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
/home/pcmoraes/Desktop/codigos/Neural-Networks/.venv/lib/python3.10/site-packages/torch/nn/modules/module.py:1532: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
/home/pcmoraes/Desktop/codigos/Neural-Networks/.venv/lib/python3.10/site-packages/torch/nn/modules/module.py:1532: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
/home/pcmoraes/Desktop/codigos/Neural-Networks/.venv/lib/python3.10/site-packages/torch/nn/modules/module.py:1532: UserWarning: Implicit dimension choice 

...

/home/pcmoraes/Desktop/codigos/Neural-Networks/.venv/lib/python3.10/site-packages/torch/nn/modules/module.py:1532: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
/home/pcmoraes/Desktop/codigos/Neural-Networks/.venv/lib/python3.10/site-packages/torch/nn/modules/module.py:1532: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
/home/pcmoraes/Desktop/codigos/Neural-Networks/.venv/lib/python3.10/site-packages/torch/nn/modules/module.py:1532: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
/home/pcmoraes/Desktop/codigos/Neural-Networks/.venv/lib/python3.10/site-packages/torch/nn/modules/module.py:1532: UserWarning: Implicit dimension choice 

Epoch 19/20 - Loss Train: 0.004474142206674243 - Accuracy Train: 0.9996841755319149 - Loss Val: 0.06676314398064279 - Accuracy Val: 0.98076171875


/home/pcmoraes/Desktop/codigos/Neural-Networks/.venv/lib/python3.10/site-packages/torch/nn/modules/module.py:1532: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
/home/pcmoraes/Desktop/codigos/Neural-Networks/.venv/lib/python3.10/site-packages/torch/nn/modules/module.py:1532: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
/home/pcmoraes/Desktop/codigos/Neural-Networks/.venv/lib/python3.10/site-packages/torch/nn/modules/module.py:1532: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
/home/pcmoraes/Desktop/codigos/Neural-Networks/.venv/lib/python3.10/site-packages/torch/nn/modules/module.py:1532: UserWarning: Implicit dimension choice 

Epoch 20/20 - Loss Train: 0.0034795000071716595 - Accuracy Train: 0.9997672872340425 - Loss Val: 0.061273212978267114 - Accuracy Val: 0.98251953125


/home/pcmoraes/Desktop/codigos/Neural-Networks/.venv/lib/python3.10/site-packages/torch/nn/modules/module.py:1532: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
/home/pcmoraes/Desktop/codigos/Neural-Networks/.venv/lib/python3.10/site-packages/torch/nn/modules/module.py:1532: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
/home/pcmoraes/Desktop/codigos/Neural-Networks/.venv/lib/python3.10/site-packages/torch/nn/modules/module.py:1532: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
/home/pcmoraes/Desktop/codigos/Neural-Networks/.venv/lib/python3.10/site-packages/torch/nn/modules/module.py:1532: UserWarning: Implicit dimension choice 

In [29]:
c2 = nn.Sequential(
    nn.Linear(32, 21),
    nn.ReLU(),
    nn.Linear(21, 10),
    nn.LogSoftmax()
)

c2 = c2.to(device)

In [30]:
criterion = nn.NLLLoss()
optimizer = optim.Adam(c2.parameters())

In [31]:
for epoch in range(20):
    running_loss_train = 0
    running_accuracy_train = 0

    for data in loader_train_codify:
        c2.train()

        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()

        outputs = c2(inputs)

        ps = torch.exp(outputs)
        _, top_class = ps.topk(1, dim=1)

        loss = criterion(outputs, labels)
        loss.backward()

        optimizer.step()

        running_loss_train += loss.item()
        running_accuracy_train += accuracy_score(labels.detach().cpu().numpy(), top_class.detach().cpu().numpy())

    running_loss_val = 0
    running_accuracy_val = 0

    for data in loader_test_codify:
        c2.eval()

        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = c2(inputs)

        ps = torch.exp(outputs)
        _, top_class = ps.topk(1, dim=1)

        loss = criterion(outputs, labels)

        running_loss_val += loss.item()
        running_accuracy_val += accuracy_score(labels.detach().cpu().numpy(), top_class.detach().cpu().numpy())

    print(f'Epoch {epoch+1}/{20} - Loss Train: {running_loss_train/len(loader_train_codify)} - Accuracy Train: {running_accuracy_train/len(loader_train_codify)} - Loss Val: {running_loss_val/len(loader_test_codify)} - Accuracy Val: {running_accuracy_val/len(loader_test_codify)}')

/home/pcmoraes/Desktop/codigos/Neural-Networks/.venv/lib/python3.10/site-packages/torch/nn/modules/module.py:1532: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
/home/pcmoraes/Desktop/codigos/Neural-Networks/.venv/lib/python3.10/site-packages/torch/nn/modules/module.py:1532: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
/home/pcmoraes/Desktop/codigos/Neural-Networks/.venv/lib/python3.10/site-packages/torch/nn/modules/module.py:1532: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
/home/pcmoraes/Desktop/codigos/Neural-Networks/.venv/lib/python3.10/site-packages/torch/nn/modules/module.py:1532: UserWarning: Implicit dimension choice 

Epoch 1/20 - Loss Train: 2.1978380111937827 - Accuracy Train: 0.3392564273049646 - Loss Val: 2.0216359943151474 - Accuracy Val: 0.5505859375


/home/pcmoraes/Desktop/codigos/Neural-Networks/.venv/lib/python3.10/site-packages/torch/nn/modules/module.py:1532: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
/home/pcmoraes/Desktop/codigos/Neural-Networks/.venv/lib/python3.10/site-packages/torch/nn/modules/module.py:1532: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
/home/pcmoraes/Desktop/codigos/Neural-Networks/.venv/lib/python3.10/site-packages/torch/nn/modules/module.py:1532: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
/home/pcmoraes/Desktop/codigos/Neural-Networks/.venv/lib/python3.10/site-packages/torch/nn/modules/module.py:1532: UserWarning: Implicit dimension choice 

Epoch 2/20 - Loss Train: 1.7338054662055158 - Accuracy Train: 0.5863419769503546 - Loss Val: 1.4341419875621795 - Accuracy Val: 0.63623046875


/home/pcmoraes/Desktop/codigos/Neural-Networks/.venv/lib/python3.10/site-packages/torch/nn/modules/module.py:1532: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
/home/pcmoraes/Desktop/codigos/Neural-Networks/.venv/lib/python3.10/site-packages/torch/nn/modules/module.py:1532: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
/home/pcmoraes/Desktop/codigos/Neural-Networks/.venv/lib/python3.10/site-packages/torch/nn/modules/module.py:1532: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
/home/pcmoraes/Desktop/codigos/Neural-Networks/.venv/lib/python3.10/site-packages/torch/nn/modules/module.py:1532: UserWarning: Implicit dimension choice 

...

/home/pcmoraes/Desktop/codigos/Neural-Networks/.venv/lib/python3.10/site-packages/torch/nn/modules/module.py:1532: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
/home/pcmoraes/Desktop/codigos/Neural-Networks/.venv/lib/python3.10/site-packages/torch/nn/modules/module.py:1532: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
/home/pcmoraes/Desktop/codigos/Neural-Networks/.venv/lib/python3.10/site-packages/torch/nn/modules/module.py:1532: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
/home/pcmoraes/Desktop/codigos/Neural-Networks/.venv/lib/python3.10/site-packages/torch/nn/modules/module.py:1532: UserWarning: Implicit dimension choice 

Epoch 19/20 - Loss Train: 0.6516586922584696 - Accuracy Train: 0.7807070035460993 - Loss Val: 0.6337863303720951 - Accuracy Val: 0.7912109375


/home/pcmoraes/Desktop/codigos/Neural-Networks/.venv/lib/python3.10/site-packages/torch/nn/modules/module.py:1532: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
/home/pcmoraes/Desktop/codigos/Neural-Networks/.venv/lib/python3.10/site-packages/torch/nn/modules/module.py:1532: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
/home/pcmoraes/Desktop/codigos/Neural-Networks/.venv/lib/python3.10/site-packages/torch/nn/modules/module.py:1532: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
/home/pcmoraes/Desktop/codigos/Neural-Networks/.venv/lib/python3.10/site-packages/torch/nn/modules/module.py:1532: UserWarning: Implicit dimension choice 

Epoch 20/20 - Loss Train: 0.6504774470278557 - Accuracy Train: 0.7807347074468085 - Loss Val: 0.6390257209539414 - Accuracy Val: 0.78916015625


/home/pcmoraes/Desktop/codigos/Neural-Networks/.venv/lib/python3.10/site-packages/torch/nn/modules/module.py:1532: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
/home/pcmoraes/Desktop/codigos/Neural-Networks/.venv/lib/python3.10/site-packages/torch/nn/modules/module.py:1532: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
